In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import pandas as pd

## Extract and Transform the data

In [ ]:
def transform_countries_df(raw):
    # This function transforms the raw data to a Dataframe with dates as 
    # rows and countries as columns
    raw_df = raw.rename(columns={"Country/Region": "country"})
    raw_df.drop(columns=["Lat", "Long"], inplace=True)
    
    # Pivot table for countries as columns and date as index
    final_df = pd.pivot_table(raw_df, columns=["country"])
    final_df = final_df.reset_index()
    final_df = final_df.rename(columns={"index": "date"})
    final_df.date = pd.to_datetime(final_df.date)
    final_df = final_df.sort_values("date").reset_index(drop=True)

    return final_df

# Extract
url = "https://raw.githubusercontent.com/Covid-19-Response-Greece/covid19-data-greece\
/master/data/all_countries/JohnsHopkinsCSSE/time_series_covid19_"
raw_conf = pd.read_csv(url + "confirmed_global.csv")
raw_death = pd.read_csv(url + "deaths_global.csv")
raw_recov = pd.read_csv(url + "recovered_global.csv")

# Transform
confirmed = transform_countries_df(raw_conf)
deaths = transform_countries_df(raw_death)
recov = transform_countries_df(raw_recov)

### Distribution array

This is the array used to distribute the new confirmed cases as recovered cases in the future

In [ ]:
pois = np.random.poisson(lam=5, size=10000)
distr_array = np.bincount(pois)/10000
distr_array = distr_array[::-1]
distr_array[0] = 0
distr_array = np.concatenate([np.array([0,0,0,0,0]), distr_array])


plt.bar(x=np.arange(len(distr_array)), height=distr_array)
plt.show()

In [ ]:
def create_data(country, start, end, distribution_array, perc):
    # This function transforms the data of the selected country in a format 
    # ready to be used by the SIR model.
    
    X_cml = np.array(confirmed[country]).astype("int")
    death = np.array(deaths[country]).astype("int")
    recovered = np.array(recov[country])
    
    # Slice the data. A reasonable starting and ending index should be used
    X_cml = X_cml[start:end]
    death = death[start:end]
    recovered = recovered[start:end]
    
    # Create new recovered data by distributing the new confirmed cases of each day 
    # as recovered cases of the coming days, using a pre-defined distribution array.
    # The new cases are reduced by some percentage, which is attributed to deaths.
    new_cases = np.diff(X_cml)
    cumul_x = np.zeros(len(new_cases))
    for i, x in enumerate(new_cases):
        r = distribution_array*x*perc
        min_indx = min(len(distribution_array), len(new_cases)-i)
        r = r[:min_indx]
        cumul_x[i:i+min_indx] += r

    temp_recov = cumul_x.astype("int")
    temp_recov = np.concatenate([np.array([0]), temp_recov.cumsum()])
    recovered = temp_recov.astype("int")
    
    return X_cml, death, recovered

In [ ]:
country_name = "US"
start_indx = 30
end_indx = 300
percent = 0.95

X_cml, death, recovered = create_data(country_name, start_indx, end_indx, distr_array, percent)

In [ ]:
print(X_cml)
print(death)
print(recovered)

## SIR model

The rest of the code is directly taken by the Chen, Lu, Chang, Liu paper, with minimal modifications. The code can be found here: https://github.com/PingEnLu/Time-dependent_SIR_COVID-19

In [ ]:
def data_spilt(data, orders, start):
    x_train = np.empty((len(data) - start - orders, orders))
    y_train = data[start + orders:]

    for i in range(len(data) - start - orders):
        x_train[i] = data[i + start:start + orders + i]

    # Exclude the day (Feb. 12, 2020) of the change of the definition of confirmed cases in Hubei China.
    #x_train = np.delete(x_train, np.s_[28 - (orders + 1) - start:28 - start], 0)
    #y_train = np.delete(y_train, np.s_[28 - (orders + 1) - start:28 - start])

    return x_train, y_train


def ridge(x, y):
    print('\nStart searching good parameters for the task...')
    parameters = {'alpha': np.arange(0, 0.100005, 0.000005).tolist(),
                  "tol": [1e-8],
                  'fit_intercept': [True, False],
                  'normalize': [True, False]}

    clf = GridSearchCV(Ridge(), parameters, n_jobs=-1, cv=5)
    clf.fit(x, y)

    print('\nResults for the parameters grid search:')
    print('Model:', clf.best_estimator_)
    print('Score:', clf.best_score_)

    return clf


In [ ]:
population = 11000000
########## data preprocess ##########
X = X_cml - recovered - death
R = recovered + death

n = np.array([population] * len(X), dtype=np.float64)

S = n - X - R

X_diff = np.array([X[:-1], X[1:]], dtype=np.float64).T
R_diff = np.array([R[:-1], R[1:]], dtype=np.float64).T

gamma = (R[1:] - R[:-1]) / X[:-1]
beta = n[:-1] * (X[1:] - X[:-1] + R[1:] - R[:-1]) / (X[:-1] * (n[:-1] - X[:-1] - R[:-1]))
R0 = beta / gamma

########## Parameters for Ridge Regression ##########
##### Orders of the two FIR filters in (12), (13) in the paper. #####
orders_beta = 10
orders_gamma = 5

##### Select a starting day for the data training in the ridge regression. #####
start_beta = 3
start_gamma = 3

########## Print Info ##########
print("\nThe latest transmission rate beta of SIR model:", beta[-1])
print("The latest recovering rate gamma of SIR model:", gamma[-1])
print("The latest basic reproduction number R0:", R0[-1])

########## Ridge Regression ##########
##### Split the data to the training set and testing set #####
x_beta, y_beta = data_spilt(beta, orders_beta, start_beta)
x_gamma, y_gamma = data_spilt(gamma, orders_gamma, start_gamma)


##### Searching good parameters #####
#clf_beta = ridge(x_beta, y_beta)
#clf_gamma = ridge(x_gamma, y_gamma)

##### Training and Testing #####
clf_beta = Ridge(alpha=0.003765, copy_X=True, fit_intercept=False, max_iter=None, normalize=True, random_state=None, solver='auto', tol=1e-08).fit(x_beta, y_beta)
clf_gamma = Ridge(alpha=0.001675, copy_X=True, fit_intercept=False, max_iter=None,normalize=True, random_state=None, solver='auto', tol=1e-08).fit(x_gamma, y_gamma)

beta_hat = clf_beta.predict(x_beta)
gamma_hat = clf_gamma.predict(x_gamma)

In [ ]:
plt.figure(1)
plt.plot(y_beta, label=r'$\beta (t)$')
plt.plot(beta_hat, label=r'$\hat{\beta}(t)$')
plt.legend()
plt.show()

plt.figure(2)
plt.plot(y_gamma, label=r'$\gamma (t)$')
plt.plot(gamma_hat, label=r'$\hat{\gamma}(t)$')
plt.legend()
plt.show()

In [ ]:
##### Parameters for the Time-dependent SIR model #####
stop_X = 1 # stopping criteria
stop_day = 350 # maximum iteration days (W in the paper)

day_count = 0
turning_point = 0

S_predict = [S[-1]]
X_predict = [X[-1]]
R_predict = [R[-1]]

predict_beta = np.array(beta[-orders_beta:]).tolist()
predict_gamma = np.array(gamma[-orders_gamma:]).tolist()
while (X_predict[-1] >= stop_X) and (day_count <= stop_day):
    if predict_beta[-1] > predict_gamma[-1]:
        turning_point += 1

    next_beta = clf_beta.predict(np.asarray([predict_beta[-orders_beta:]]))[0]
    next_gamma = clf_gamma.predict(np.asarray([predict_gamma[-orders_gamma:]]))[0]
    #next_gamma = predict_gamma[-1]
    if next_beta < 0:
        next_beta = 0
    if next_gamma < 0:
        next_gamma = 0
        
    #print("beta", next_beta)
    #print("gamma", next_gamma)

    predict_beta.append(next_beta)
    predict_gamma.append(next_gamma)

    next_S = ((-predict_beta[-1] * S_predict[-1] *
               X_predict[-1]) / n[-1]) + S_predict[-1]
    next_X = ((predict_beta[-1] * S_predict[-1] * X_predict[-1]) /
              n[-1]) - (predict_gamma[-1] * X_predict[-1]) + X_predict[-1]
    next_R = (predict_gamma[-1] * X_predict[-1]) + R_predict[-1]

    S_predict.append(next_S)
    X_predict.append(next_X)
    R_predict.append(next_R)

    day_count += 1


In [ ]:
plt.plot(range(len(predict_beta)), predict_beta, label=r'$\hat{\beta}(t)$')
plt.legend()
plt.show()

plt.plot(range(len(predict_gamma)), predict_gamma, label=r'$\hat{\gamma}(t)$')
plt.legend()
plt.show()

In [ ]:
########## Print Info ##########
print('\nConfirmed cases tomorrow:', np.rint(X_predict[1] + R_predict[1]))
print('Infected persons tomorrow:', np.rint(X_predict[1]))
print('Recovered + Death persons tomorrow:', np.rint(R_predict[1]))

print('\nEnd day:', day_count)
print('Confirmed cases on the end day:', np.rint(X_predict[-2] + R_predict[-2]))

print('\nTuring point:', turning_point)

In [ ]:
########## Plot the time evolution of the time-dependent SIR model ##########
plt.figure(figsize=(15,10))
plt.plot(range(len(X) - 1, len(X) - 1 + len(X_predict)), X_predict, '*-', label=r'$\hat{X}(t)$', color='darkorange')
plt.plot(range(len(X) - 1, len(X) - 1 + len(X_predict)), R_predict, '*-', label=r'$\hat{R}(t)$', color='limegreen')
plt.plot(range(len(X)), np.array(X)+np.array(R), 'o--', label='Total', color='cornflowerblue')
plt.plot(range(len(X) - 1, len(X) - 1 + len(X_predict)), np.array(X_predict) + np.array(R_predict), \
         '*-', label='Total predicted', color='blue')
plt.plot(range(len(X)), X, 'o--', label=r'$X(t)$', color='chocolate')
plt.plot(range(len(X)), R, 'o--', label=r'$R(t)$', color='darkgreen')
plt.xlabel('Day')
#plt.ylabel('Person')
plt.title('Time evolution of the time-dependent SIR model.')

plt.legend()

plt.show()

In [ ]:
print(int(R_predict[-1]))